In [1]:
import numpy as np
import pandas as pd
import gc
import os
import time
import random
from tqdm.auto import tqdm
import datetime

In [2]:
Nval = 50
#Nsub = 12 # 提出用
Nsub = Nval
#val_start_date = '2020-09-09' # 他のnotebookに日付の書き方揃える
val_start_date = '2020-09-16' #prod

In [3]:
from datetime import datetime, date, timedelta
val_end_date = pd.to_datetime(val_start_date) + timedelta(days=7)

In [4]:
val_end_date = str(val_end_date.strftime('%Y-%m-%d'))
print(val_start_date, val_end_date)

2020-09-16 2020-09-23


In [5]:
def visualize_df(df):
    print(df.shape)
    display(df.head())

In [6]:
# データの読み込み
dir = 'h-and-m-personalized-fashion-recommendations/'
path = '../input/' + dir 

#customers = pd.read_csv(path + 'customers.csv')
#articles = pd.read_csv(path + 'articles.csv')
transactions_train = pd.read_csv(path + 'transactions_train.csv', usecols=['t_dat', 'customer_id', 'article_id', 'price'])
transactions_train['customer_id'] = transactions_train['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
transactions_train['article_id'] = transactions_train['article_id'].astype('int32')

In [7]:
transactions_train

,t_dat,customer_id,article_id,price
0,2018-09-20,-6846340800584936,663713001,0.050831
1,2018-09-20,-6846340800584936,541518023,0.030492
2,2018-09-20,-8334631767138808638,505221004,0.015237
3,2018-09-20,-8334631767138808638,685687003,0.016932
4,2018-09-20,-8334631767138808638,685687004,0.016932
...,...,...,...,...
31788319,2020-09-22,4685485978980270934,929511001,0.059305
31788320,2020-09-22,4685485978980270934,891322004,0.042356
31788321,2020-09-22,3959348689921271969,918325001,0.043203
31788322,2020-09-22,-8639340045377511665,833459002,0.006763


In [8]:
## https://www.kaggle.com/code/lichtlab/h-m-data-deep-dive-chap-1-understand-article
## 現在売れてない商品は除く
#dfagg1 = transactions_train.groupby(['article_id'])[['price']].sum().reset_index().rename(columns={'price': 'article_ttl_sales'})
#dfagg2 = transactions_train[transactions_train['t_dat'] < '2019-09-01'].groupby(['article_id'])[['price']].sum().reset_index().rename(columns={'price': 'before_201909_article_ttl_sales'})
#dfagg3 = transactions_train['2019-09-01' < transactions_train['t_dat']].groupby(['article_id'])[['price']].sum().reset_index().rename(columns={'price': 'after_201909_article_ttl_sales'})
#
#dfagg1 = pd.merge(dfagg1, dfagg2, on='article_id')
#dfagg1 = pd.merge(dfagg1, dfagg3, on='article_id')
#del dfagg2, dfagg3; gc.collect()
#dfagg1['before_201909_article_ttl_sales_ratio'] = dfagg1['before_201909_article_ttl_sales'] / dfagg1['article_ttl_sales'] * 100
#dfagg1['not_for_sales_flg'] = (dfagg1['before_201909_article_ttl_sales_ratio'] > 99.9).astype(int)
#dfagg1

In [9]:
## 使えるarticle_idだけ残す
#dfagg1 = dfagg1.query("not_for_sales_flg == 0")['article_id'].reset_index(drop=True)
#dfagg1

In [10]:
sub = pd.read_csv(path + 'sample_submission.csv')
sub['customer_id'] = sub['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
del sub['prediction']; gc.collect()
visualize_df(sub)

(1371980, 1)


,customer_id
0,6883939031699146327
1,-7200416642310594310
2,-6846340800584936
3,-94071612138601410
4,-283965518499174310


In [11]:
visualize_df(transactions_train)

(31788324, 4)


,t_dat,customer_id,article_id,price
0,2018-09-20,-6846340800584936,663713001,0.050831
1,2018-09-20,-6846340800584936,541518023,0.030492
2,2018-09-20,-8334631767138808638,505221004,0.015237
3,2018-09-20,-8334631767138808638,685687003,0.016932
4,2018-09-20,-8334631767138808638,685687004,0.016932


In [12]:
# transactions_train['t_dat'].unique()[-7:]

# array(['2020-09-16', '2020-09-17', '2020-09-18', '2020-09-19',
#       '2020-09-20', '2020-09-21', '2020-09-22'], dtype=object)

In [13]:
train_data = transactions_train.query(f"t_dat < '{val_end_date}'").reset_index(drop=True)
valid_data = transactions_train.query(f"'2020-09-16' <= t_dat").reset_index(drop=True) # fix
visualize_df(train_data)
visualize_df(valid_data)

(31788324, 4)


,t_dat,customer_id,article_id,price
0,2018-09-20,-6846340800584936,663713001,0.050831
1,2018-09-20,-6846340800584936,541518023,0.030492
2,2018-09-20,-8334631767138808638,505221004,0.015237
3,2018-09-20,-8334631767138808638,685687003,0.016932
4,2018-09-20,-8334631767138808638,685687004,0.016932


(240311, 4)


,t_dat,customer_id,article_id,price
0,2020-09-16,8055340703569116761,786022008,0.048441
1,2020-09-16,8055340703569116761,913272003,0.032288
2,2020-09-16,8055340703569116761,889669006,0.056508
3,2020-09-16,2563761487478961358,237347060,0.033881
4,2020-09-16,2563761487478961358,562245001,0.013542


In [14]:
# メモリ節約
del transactions_train
gc.collect()  

3

### Simple Collaborative Filtering

In [15]:
# t_datを週ごとの値に変換したものを追加
# customer_id, article_idとtimeのみ残す
# t_datを1週間進めたdfを作る
# cross　joinしてt_dat, customer_idごとにまとめる

#### wbywの場合

In [16]:
# t_datをobjectからtimestamp型に変換
train_data['t_dat'] = pd.to_datetime(train_data['t_dat'])
train_data['t_dat_1week'] = train_data['t_dat'] + timedelta(days=7)

In [17]:
train_data.head()

,t_dat,customer_id,article_id,price,t_dat_1week
0,2018-09-20,-6846340800584936,663713001,0.050831,2018-09-27
1,2018-09-20,-6846340800584936,541518023,0.030492,2018-09-27
2,2018-09-20,-8334631767138808638,505221004,0.015237,2018-09-27
3,2018-09-20,-8334631767138808638,685687003,0.016932,2018-09-27
4,2018-09-20,-8334631767138808638,685687004,0.016932,2018-09-27


In [18]:
# 週ごとのカラムを追加
train_data['week'] = train_data['t_dat'].dt.to_period("W")
train_data['week_future'] = train_data['t_dat_1week'].dt.to_period("W")
# train_cf = train_data.set_index('t_dat')[['customer_id', 'article_id']]

In [19]:
train_cf = train_data[['week', 'week_future', 'customer_id', 'article_id']]
train_cf

,week,week_future,customer_id,article_id
0,2018-09-17/2018-09-23,2018-09-24/2018-09-30,-6846340800584936,663713001
1,2018-09-17/2018-09-23,2018-09-24/2018-09-30,-6846340800584936,541518023
2,2018-09-17/2018-09-23,2018-09-24/2018-09-30,-8334631767138808638,505221004
3,2018-09-17/2018-09-23,2018-09-24/2018-09-30,-8334631767138808638,685687003
4,2018-09-17/2018-09-23,2018-09-24/2018-09-30,-8334631767138808638,685687004
...,...,...,...,...
31788319,2020-09-21/2020-09-27,2020-09-28/2020-10-04,4685485978980270934,929511001
31788320,2020-09-21/2020-09-27,2020-09-28/2020-10-04,4685485978980270934,891322004
31788321,2020-09-21/2020-09-27,2020-09-28/2020-10-04,3959348689921271969,918325001
31788322,2020-09-21/2020-09-27,2020-09-28/2020-10-04,-8639340045377511665,833459002


In [20]:
train_future = train_cf
train_future = train_future.drop('week', axis='columns')
train_future = train_future.rename(columns={'week_future':'week'})

In [21]:
## join
joined_train =  pd.merge(train_cf, train_future, on=['customer_id','week'], how='inner')

In [22]:
del train_future
del train_cf
gc.collect() 

36

#### fromとtoを結合後

In [23]:
joined_train

,week,week_future,customer_id,article_id_x,article_id_y
0,2018-09-24/2018-09-30,2018-10-01/2018-10-07,-6846340800584936,663713001,663713001
1,2018-09-24/2018-09-30,2018-10-01/2018-10-07,-6846340800584936,663713001,541518023
2,2018-09-24/2018-09-30,2018-10-01/2018-10-07,-1654819258089153483,380609010,625939013
3,2018-09-24/2018-09-30,2018-10-01/2018-10-07,-1654819258089153483,380609010,625939013
4,2018-09-24/2018-09-30,2018-10-01/2018-10-07,-1654819258089153483,380609010,712838007
...,...,...,...,...,...
31649731,2020-09-21/2020-09-27,2020-09-28/2020-10-04,4685485978980270934,891322004,918892001
31649732,2020-09-21/2020-09-27,2020-09-28/2020-10-04,4685485978980270934,891322004,918892001
31649733,2020-09-21/2020-09-27,2020-09-28/2020-10-04,4685485978980270934,891322004,879088001
31649734,2020-09-21/2020-09-27,2020-09-28/2020-10-04,4685485978980270934,891322004,894780001


In [24]:
# 使わなくなったカラムは消す
joined_train = joined_train[['article_id_x', 'article_id_y']]

In [25]:
joined_train = joined_train.rename(columns={'article_id_x':'from_article_id', 'article_id_y':'to_article_id'})
joined_train

,from_article_id,to_article_id
0,663713001,663713001
1,663713001,541518023
2,380609010,625939013
3,380609010,625939013
4,380609010,712838007
...,...,...
31649731,891322004,918892001
31649732,891322004,918892001
31649733,891322004,879088001
31649734,891322004,894780001


In [26]:
## 使われてないarticel_idがtoのものを削除
#joined_train = pd.merge(joined_train, dfagg1, left_on='to_article_id', right_on='article_id', how='inner')
#joined_train = joined_train.drop('article_id', axis='columns')
#del dfagg1; gc.collect() 

In [27]:
group = joined_train.groupby(['from_article_id', 'to_article_id'])

In [28]:
del joined_train
gc.collect() 

0

In [29]:
group_df = group.size().reset_index(name='counts')
group_df = group_df.sort_values(['from_article_id', 'counts'],ascending=False)
group_df

,from_article_id,to_article_id,counts
20805181,956217002,926166001,4
20805125,956217002,746070007,2
20805127,956217002,771759011,2
20805133,956217002,815848001,2
20805134,956217002,855198005,2
...,...,...,...
3407,108775015,874465005,1
3408,108775015,878013004,1
3409,108775015,880839006,1
3410,108775015,899247005,1


### Threthold

In [30]:
group_df['counts'].describe()

count    2.080519e+07
mean     1.521242e+00
std      1.033580e+01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      4.560100e+04
Name: counts, dtype: float64

In [31]:
# Thretholdの設定 10以上とか
group_df = group_df.query('counts>10')
group_df['counts'].describe()

count    69323.000000
mean        23.154797
std        176.921391
min         11.000000
25%         12.000000
50%         15.000000
75%         21.000000
max      45601.000000
Name: counts, dtype: float64

In [32]:
group2 = group_df.groupby('from_article_id')['to_article_id'].apply(list).reset_index(name='rec_list')

In [33]:
# メモリ節約
del group_df
del group
gc.collect() 

0

In [34]:
group2 = group2.set_index('from_article_id')
group2.head()

,rec_list
from_article_id,
108775015,"[108775015, 565379001, 108775044, 372860001, 6..."
108775044,"[108775044, 108775015, 664074001, 479167001, 3..."
110065011,"[110065011, 687260002]"
111565001,"[111565001, 700910001, 228257001, 529012025, 5..."
111586001,"[111586001, 436261001, 111593001, 158340001, 6..."


In [35]:
# ちゃんとsortされてlistになってるか確認
#group2.loc[956217002]

In [36]:
group2['rec_list'].apply(len).describe()

count    17713.000000
mean         3.913679
std          7.922570
min          1.000000
25%          1.000000
50%          2.000000
75%          4.000000
max        299.000000
Name: rec_list, dtype: float64

In [37]:
group2['len'] = group2['rec_list'].apply(len)

In [38]:
group2.head()

,rec_list,len
from_article_id,,
108775015,"[108775015, 565379001, 108775044, 372860001, 6...",33
108775044,"[108775044, 108775015, 664074001, 479167001, 3...",34
110065011,"[110065011, 687260002]",2
111565001,"[111565001, 700910001, 228257001, 529012025, 5...",11
111586001,"[111586001, 436261001, 111593001, 158340001, 6...",30


In [39]:
# 長さがN以上のものは上位Nにまるめる
group2['rec_list'] = group2['rec_list'].apply(lambda x: x[:Nval])

In [40]:
group2['len'] = group2['rec_list'].apply(len)
group2['rec_list'].apply(len).describe()

count    17713.000000
mean         3.778637
std          6.154363
min          1.000000
25%          1.000000
50%          2.000000
75%          4.000000
max         50.000000
Name: rec_list, dtype: float64

In [41]:
group2

,rec_list,len
from_article_id,,
108775015,"[108775015, 565379001, 108775044, 372860001, 6...",33
108775044,"[108775044, 108775015, 664074001, 479167001, 3...",34
110065011,"[110065011, 687260002]",2
111565001,"[111565001, 700910001, 228257001, 529012025, 5...",11
111586001,"[111586001, 436261001, 111593001, 158340001, 6...",30
...,...,...
941005003,[941005003],1
941005005,"[751664001, 941005005]",2
944506001,"[909924002, 944506001]",2


### analyse user behaviour

In [42]:
# userごとに直近の購入商品をいくつか集める

In [43]:
train_data.head()

,t_dat,customer_id,article_id,price,t_dat_1week,week,week_future
0,2018-09-20,-6846340800584936,663713001,0.050831,2018-09-27,2018-09-17/2018-09-23,2018-09-24/2018-09-30
1,2018-09-20,-6846340800584936,541518023,0.030492,2018-09-27,2018-09-17/2018-09-23,2018-09-24/2018-09-30
2,2018-09-20,-8334631767138808638,505221004,0.015237,2018-09-27,2018-09-17/2018-09-23,2018-09-24/2018-09-30
3,2018-09-20,-8334631767138808638,685687003,0.016932,2018-09-27,2018-09-17/2018-09-23,2018-09-24/2018-09-30
4,2018-09-20,-8334631767138808638,685687004,0.016932,2018-09-27,2018-09-17/2018-09-23,2018-09-24/2018-09-30


In [44]:
train_data = train_data.sort_values(['customer_id', 't_dat'],ascending=False)
group3 = train_data.groupby('customer_id')['article_id'].apply(list).reset_index(name='seed_list')

In [45]:
group3['seed_list'] = group3['seed_list'].apply(lambda x: x[:3])

In [46]:
group3['top_seed'] = group3['seed_list'].apply(lambda x: x[0])
#group3['second_seed'] = group3['seed_list'].apply(lambda x: x[1])
#group3['third_seed'] = group3['seed_list'].apply(lambda x: x[2])

In [47]:
group3

,customer_id,seed_list,top_seed
0,-9223352921020755230,"[673396002, 812167004, 849493006]",673396002
1,-9223343869995384291,"[910601003, 908292002, 903926002]",910601003
2,-9223321797620987725,"[580600006, 610776035]",580600006
3,-9223319430705797669,"[470985003, 562245001, 646592007]",470985003
4,-9223308614576639426,[750423005],750423005
...,...,...,...
1362276,9223319300843860958,"[821663003, 640735005]",821663003
1362277,9223333063893176977,"[658030020, 452818021, 689365016]",658030020
1362278,9223345314868180224,"[552018006, 549403003, 625672005]",552018006
1362279,9223357421094039679,"[565379022, 565379022, 565379022]",565379022


In [48]:
joined_group3 =  pd.merge(group3, group2, left_on='top_seed', right_on='from_article_id', how='left').fillna('')

In [49]:
## rec_listがない場合はsecond, thirdでjoinしたい

In [50]:
# メモリ節約
del group2
del group3
del train_data
gc.collect() 

0

In [51]:
joined_group3 = joined_group3[['customer_id', 'rec_list', 'len']]

In [52]:
joined_group3

,customer_id,rec_list,len
0,-9223352921020755230,"[673396002, 589748001, 706016001, 448509014, 7...",30.0
1,-9223343869995384291,"[910601003, 910601002, 910601001, 751471001, 8...",8.0
2,-9223321797620987725,,
3,-9223319430705797669,"[470985003, 875469002, 803757012, 817353007, 6...",5.0
4,-9223308614576639426,"[750423005, 787880003, 777504001]",3.0
...,...,...,...
1362276,9223319300843860958,"[821663003, 821663005, 179208001]",3.0
1362277,9223333063893176977,"[658030020, 556260002, 658030011, 658030006, 6...",20.0
1362278,9223345314868180224,,
1362279,9223357421094039679,"[565379022, 565379002, 565379023, 565379032, 3...",12.0


### ランキング csvの読み込み

In [53]:
# 事前に出力しておいたrankingデータの読み込み
simple_ranking = pd.read_csv(f'./simple_ranking_{val_start_date}_{Nval}.csv')

In [54]:
simple_ranking = simple_ranking.rename(columns={'prediction':'ranking'})

In [55]:
simple_ranking['ranking'].apply(len).describe()

count    1371980.0
mean         499.0
std            0.0
min          499.0
25%          499.0
50%          499.0
75%          499.0
max          499.0
Name: ranking, dtype: float64

In [56]:
#simple_ranking['ranking'] = simple_ranking['ranking'].apply(lambda x: x[:N])

In [57]:
simple_ranking['ranking'] = simple_ranking['ranking'].str.split(' ')
simple_ranking

,customer_id,ranking
0,-7200416642310594310,"[924243001, 918522001, 866731001, 448509014, 9..."
1,-6846340800584936,"[924243001, 918522001, 866731001, 448509014, 9..."
2,-2477030216582433370,"[924243001, 918522001, 866731001, 448509014, 9..."
3,-3402710142238164438,"[924243001, 918522001, 866731001, 448509014, 9..."
4,5669685583838440396,"[924243001, 918522001, 866731001, 448509014, 9..."
...,...,...
1371975,8959640756044895752,"[924243001, 930380001, 924243002, 928206001, 9..."
1371976,8536186189414498976,"[924243001, 930380001, 924243002, 928206001, 9..."
1371977,-1244504865876165278,"[924243001, 930380001, 924243002, 928206001, 9..."
1371978,5823894334162843119,"[924243001, 930380001, 924243002, 928206001, 9..."


In [58]:
simple_ranking['ranking'] = simple_ranking['ranking'].apply(lambda x: x[:Nval])
simple_ranking['ranking'].apply(len).describe()

count    1371980.0
mean          50.0
std            0.0
min           50.0
25%           50.0
50%           50.0
75%           50.0
max           50.0
Name: ranking, dtype: float64

### join ranking

In [59]:
joined_group3 = joined_group3[['customer_id','rec_list']]
joined_group3

,customer_id,rec_list
0,-9223352921020755230,"[673396002, 589748001, 706016001, 448509014, 7..."
1,-9223343869995384291,"[910601003, 910601002, 910601001, 751471001, 8..."
2,-9223321797620987725,
3,-9223319430705797669,"[470985003, 875469002, 803757012, 817353007, 6..."
4,-9223308614576639426,"[750423005, 787880003, 777504001]"
...,...,...
1362276,9223319300843860958,"[821663003, 821663005, 179208001]"
1362277,9223333063893176977,"[658030020, 556260002, 658030011, 658030006, 6..."
1362278,9223345314868180224,
1362279,9223357421094039679,"[565379022, 565379002, 565379023, 565379032, 3..."


In [60]:
joined_group3 = pd.merge(simple_ranking, joined_group3, on='customer_id', how='left').fillna('')

In [61]:
del simple_ranking; gc.collect()
joined_group3

,customer_id,ranking,rec_list
0,-7200416642310594310,"[924243001, 918522001, 866731001, 448509014, 9...",
1,-6846340800584936,"[924243001, 918522001, 866731001, 448509014, 9...","[805000001, 794321007]"
2,-2477030216582433370,"[924243001, 918522001, 866731001, 448509014, 9...","[738133005, 738133007, 738133006, 158340001]"
3,-3402710142238164438,"[924243001, 918522001, 866731001, 448509014, 9...","[719530003, 719530004, 706016001, 719530007, 5..."
4,5669685583838440396,"[924243001, 918522001, 866731001, 448509014, 9...",
...,...,...,...
1371975,8959640756044895752,"[924243001, 930380001, 924243002, 928206001, 9...","[160442010, 160442007, 160442043, 355072001, 3..."
1371976,8536186189414498976,"[924243001, 930380001, 924243002, 928206001, 9...",
1371977,-1244504865876165278,"[924243001, 930380001, 924243002, 928206001, 9...",
1371978,5823894334162843119,"[924243001, 930380001, 924243002, 928206001, 9...",


In [62]:
# 重複排除, このやり方だと時間かかる
joined_group3['ranking'] = joined_group3['ranking'].map(lambda x: [i for i in x if i not in joined_group3['rec_list']] )

In [63]:
# rec_listの空文字はlistにしてから結合する必要がある
joined_group3['rec_list'] = joined_group3.apply(lambda x: (list(x['rec_list'])+x['ranking'])[:Nsub], axis = 1)
joined_group3 = joined_group3.drop(columns='ranking')
gc.collect() 
joined_group3

,customer_id,rec_list
0,-7200416642310594310,"[924243001, 918522001, 866731001, 448509014, 9..."
1,-6846340800584936,"[805000001, 794321007, 924243001, 918522001, 8..."
2,-2477030216582433370,"[738133005, 738133007, 738133006, 158340001, 9..."
3,-3402710142238164438,"[719530003, 719530004, 706016001, 719530007, 5..."
4,5669685583838440396,"[924243001, 918522001, 866731001, 448509014, 9..."
...,...,...
1371975,8959640756044895752,"[160442010, 160442007, 160442043, 355072001, 3..."
1371976,8536186189414498976,"[924243001, 930380001, 924243002, 928206001, 9..."
1371977,-1244504865876165278,"[924243001, 930380001, 924243002, 928206001, 9..."
1371978,5823894334162843119,"[924243001, 930380001, 924243002, 928206001, 9..."


In [64]:
joined_group3['rec_list'].apply(len)

0          50
1          50
2          50
3          50
4          50
           ..
1371975    50
1371976    50
1371977    50
1371978    50
1371979    50
Name: rec_list, Length: 1371980, dtype: int64

In [65]:
gc.collect()

0

In [66]:
joined_group3

,customer_id,rec_list
0,-7200416642310594310,"[924243001, 918522001, 866731001, 448509014, 9..."
1,-6846340800584936,"[805000001, 794321007, 924243001, 918522001, 8..."
2,-2477030216582433370,"[738133005, 738133007, 738133006, 158340001, 9..."
3,-3402710142238164438,"[719530003, 719530004, 706016001, 719530007, 5..."
4,5669685583838440396,"[924243001, 918522001, 866731001, 448509014, 9..."
...,...,...
1371975,8959640756044895752,"[160442010, 160442007, 160442043, 355072001, 3..."
1371976,8536186189414498976,"[924243001, 930380001, 924243002, 928206001, 9..."
1371977,-1244504865876165278,"[924243001, 930380001, 924243002, 928206001, 9..."
1371978,5823894334162843119,"[924243001, 930380001, 924243002, 928206001, 9..."


In [67]:
joined_group3['rec_list'] = joined_group3['rec_list'].map(lambda x:  ' '.join(map(str, x))) 
joined_group3 = joined_group3.rename(columns={'rec_list':'valid_pred'})

In [68]:
gc.collect() 

0

In [69]:
joined_group3

,customer_id,valid_pred
0,-7200416642310594310,924243001 918522001 866731001 448509014 909370...
1,-6846340800584936,805000001 794321007 924243001 918522001 866731...
2,-2477030216582433370,738133005 738133007 738133006 158340001 924243...
3,-3402710142238164438,719530003 719530004 706016001 719530007 572797...
4,5669685583838440396,924243001 918522001 866731001 448509014 909370...
...,...,...
1371975,8959640756044895752,160442010 160442007 160442043 355072001 373506...
1371976,8536186189414498976,924243001 930380001 924243002 928206001 910601...
1371977,-1244504865876165278,924243001 930380001 924243002 928206001 910601...
1371978,5823894334162843119,924243001 930380001 924243002 928206001 910601...


### Validation data

In [70]:
valid_unq = valid_data.groupby('customer_id')['article_id'].apply(list).reset_index()
del valid_data; gc.collect()
valid_unq['valid_true'] = valid_unq['article_id'].map(lambda x: ' '.join(map(str, x)))
visualize_df(valid_unq)

(68984, 3)


,customer_id,article_id,valid_true
0,-9223100958908512198,"[785034009, 673677027, 673677024]",785034009 673677027 673677024
1,-9223002534477110135,[793699001],793699001
2,-9222810895170663723,"[892309001, 903306004]",892309001 903306004
3,-9221924794303263774,"[888343003, 892857002, 708138013]",888343003 892857002 708138013
4,-9221811157628158522,[891663002],891663002


### merge train and valid

In [71]:
merged = pd.merge(joined_group3, valid_unq, on='customer_id', how='left').fillna('')
del joined_group3, valid_unq; gc.collect()

0

In [72]:
merged

,customer_id,valid_pred,article_id,valid_true
0,-7200416642310594310,924243001 918522001 866731001 448509014 909370...,,
1,-6846340800584936,805000001 794321007 924243001 918522001 866731...,,
2,-2477030216582433370,738133005 738133007 738133006 158340001 924243...,,
3,-3402710142238164438,719530003 719530004 706016001 719530007 572797...,,
4,5669685583838440396,924243001 918522001 866731001 448509014 909370...,,
...,...,...,...,...
1371975,8959640756044895752,160442010 160442007 160442043 355072001 373506...,,
1371976,8536186189414498976,924243001 930380001 924243002 928206001 910601...,,
1371977,-1244504865876165278,924243001 930380001 924243002 928206001 910601...,,
1371978,5823894334162843119,924243001 930380001 924243002 928206001 910601...,,


In [73]:
del merged['article_id']; gc.collect()
merged.head()

,customer_id,valid_pred,valid_true
0,-7200416642310594310,924243001 918522001 866731001 448509014 909370...,
1,-6846340800584936,805000001 794321007 924243001 918522001 866731...,
2,-2477030216582433370,738133005 738133007 738133006 158340001 924243...,
3,-3402710142238164438,719530003 719530004 706016001 719530007 572797...,
4,5669685583838440396,924243001 918522001 866731001 448509014 909370...,


In [74]:
merged_small = merged.query('valid_true!=""')

In [75]:
merged_small

,customer_id,valid_pred,valid_true
37,8215006181404469139,624486001 624486002 624486003 706016002 624486...,624486001
49,-7604547624187760215,757926001 711053003 413029008 861442003 924243...,757926001 788575004 640021019
53,-7555520646521469595,924243001 918522001 866731001 448509014 909370...,874110016
91,-7641768468830472284,158340001 436261001 111586001 111593001 792515...,903762001 879189005 158340001 867966009 915529...
112,-6733796282925309862,924243001 918522001 866731001 448509014 909370...,786304008
...,...,...,...
1371911,237419416267454517,909370001 924243001 915526001 910601003 893059...,924243001 873679001 910601002
1371917,3959348689921271969,924243001 930380001 924243002 928206001 910601...,918325001
1371939,-8993704402252896919,909370001 924243001 915526001 910601003 893059...,924243001
1371954,4987941324272823374,898918002 917843001 828050002 664074072 898918...,898918002 855198005 902163001 913688001 874465...


In [76]:
merged_small['valid_true'].apply(len).describe()

count    68984.000000
mean        33.835759
std         35.351441
min          9.000000
25%          9.000000
50%         19.000000
75%         39.000000
max       1039.000000
Name: valid_true, dtype: float64

### Function

In [77]:
# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/306007
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py


def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [78]:
# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/306007
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py


def recallk(actual, predicted, k=10):
  
    if len(predicted)>k:
        predicted = predicted[:k]

    num_hits = 0.0
 
    for i,a in enumerate(actual):
        if a in predicted and a not in actual[:i]:
            num_hits += 1.0

    if not actual:
        return 0.0

    return num_hits / min(len(actual), k)


def mrecallk(actual, predicted, k=10):

    return np.mean([recallk(a,p,k) for a,p in zip(actual, predicted)])

In [79]:
tqdm.pandas()

mapk(merged_small['valid_true'].map(lambda x: x.split()), merged_small['valid_pred'].map(lambda x: x.split()), k=12)
# week by week 0.011140732679736226
# to: last week, from: others 0.00782014766757182
# to: last, from: 1 momth 0.006115032090255837

# ranking穴埋め
# rankingのみ 0.008645699012343824
# wbw 0.012118011333393222
# wbw + th>10: 0.014159445419290496
# to: last, from: 1 momth 0.003676811944912249
# to: last, from: 1 momth + th>10 : 0.010828227012764523
# to: last week, from: others 0.00870956577336267
# to: last week, from: others  + th>10 0.010946752128587353
# new 0.013703949842627761
# new+remove: 0.010583134597773845


0.2292538374496016

In [80]:
mrecallk(merged_small['valid_true'].map(lambda x: x.split()), merged_small['valid_pred'].map(lambda x: x.split()), k=5)
# week by week : 0.015262959526846804
# to: last week, from: others 0.012488644709884417
# to: last, from: 1 momth 0.0095742007808574

# ranking穴埋め
# rankingのみ 0.014520128496282925
# wbw 0.016654527029166803
# wbw + th>10: 0.02062814706906926
# to: last, from: 1 momth 0.00807206722192313
# to: last, from: 1 momth + th>10 : 0.017264939671279726
# to: last week, from: others 0.013864257740338429
# to: last week, from: others  + th>10 0.017488203439212084

# new wbw + th>10: 0.0201575727704975
# new+remove: 0.016397763732652983

0.25792405968533766

In [81]:
mrecallk(merged_small['valid_true'].map(lambda x: x.split()), merged_small['valid_pred'].map(lambda x: x.split()), k=12)
# week by week : 0.01932784589487593
# to: last week, from: others 0.019874239612874656
# to: last, from: 1 momth 0.014448328587476795

# ranking穴埋め
# rankingのみ 0.025815260523814648
# wbw 0.02123677863694331
# wbw + th>10: 0.032301029444915814
# to: last, from: 1 momth 0.01646668081929506
# to: last, from: 1 momth + th>10 : 0.028598971245121927
# to: last week, from: others 0.022308354127003778
# to: last week, from: others  + th>10 0.029319012491099767

# new wbw + th>10: 0.0316992153606441
# new+remove: 0.02783908661757148

0.26094793673217653

In [82]:
mrecallk(merged_small['valid_true'].map(lambda x: x.split()), merged_small['valid_pred'].map(lambda x: x.split()), k=20)
# week by week : 0.02257593075135928
# to: last week, from: others 0.026311201687301257
# to: last, from: 1 momth 0.018838052674924232

# ranking穴埋め
# rankingのみ 0.03797998284689133
# wbw 0.025000857027370597
# wbw + th>10: 0.04350661359738316

# to: last, from: 1 momth 0.024878600349887987
# to: last, from: 1 momth + th>10 : 0.040501679138651214
# to: last week, from: others 0.02990506652374502
# to: last week, from: others  + th>10 0.04073618499720119

# new wbw + th>10: 0.04289436483459848
# new remove: 0.0.039404877884636855

0.2682391792227964

In [83]:
mrecallk(merged_small['valid_true'].map(lambda x: x.split()), merged_small['valid_pred'].map(lambda x: x.split()), k=50)
# week by week : 0.029968311213853332
# to: last week, from: others 0.045209030309751125
# to: last, from: 1 momth 0.029489627092570865

# ranking穴埋め
# rankingのみ 0.07263431758086934
# wbw 0.03439503464484325
# wbw + th>10: 0.07790532307586577
# to: last, from: 1 momth 0.05102074236713631
# to: last, from: 1 momth + th>10 : 0.07474854400839524
# to: last week, from: others 0.05306972787354015
# to: last week, from: others  + th>10 0.07461423895697561

# new wbw + th>10: 0.07703617148752748
# new+remove: 0.07356823386454821

0.29338384223648184

In [84]:
mrecallk(merged_small['valid_true'].map(lambda x: x.split()), merged_small['valid_pred'].map(lambda x: x.split()), k=100)
# week by week : 0.03837003067670086
# to: last week, from: others 0.06662652084060305
# to: last, from: 1 momth 0.03967563687281079

# new wbw + th>10:  

0.29338125904910245

### submit

In [85]:
sub = merged[['customer_id', 'valid_pred']].copy()
sub.columns = ['customer_id', 'prediction']
print(sub.shape)

#sub.to_csv('submission.csv', index=False)
sub.to_csv(f'cf_{val_start_date}_{Nval}.csv', index=False)

(1371980, 2)
